In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


input_data = pd.read_excel('/Users/dylan/DataBase/signal_process/50ETF-signalout copy.xlsx')
signal_input = pd.read_excel('/Users/dylan/DataBase/signal_process/signal_result-30 copy.xlsx')

input_data.dropna(axis = 1, inplace= True)

# 加权选择

signal_input_quantile = signal_input[['annual_return_sp','win_rate_sp','pc_ratio_sp']].quantile([0.25,0.5,0.75])
for col_name in ['annual_return_sp','win_rate_sp','pc_ratio_sp']:
    def get_weight_group(x) :
        if x < signal_input_quantile[col_name].loc[0.25]: 
            return 1
        elif signal_input_quantile[col_name].loc[0.25] < x  and  x < signal_input_quantile[col_name].loc[0.5]: 
            return 2 
        elif signal_input_quantile[col_name].loc[0.5] < x and  x < signal_input_quantile[col_name].loc[0.75] : 
            return 3         
        else : 
            return 4 
    new_weighte_col_name = col_name + '_weight'
    signal_input[new_weighte_col_name] = signal_input[col_name].map(get_weight_group)

signal_input['Max_method'] = signal_input[['annual_return_sp_weight','win_rate_sp_weight','pc_ratio_sp_weight']].sum(axis = 1)

signal_input['Max_method'].sort_values(ascending = False)


93      12
519     12
113     12
114     12
115     12
        ..
2379     3
2380     3
2381     3
2382     3
2144     3
Name: Max_method, Length: 2654, dtype: int64

In [4]:

""" 当期相同则计算上期method
if len(signal_input_filter['Max_method']) > 3:       
    signal_input_filter.sort_values(by=['model','start_date'],inplace=True) 
    signal_input_filter['Last_method'] = signal_input_filter['Max_method'].shift(1) 
    signal_output = signal_input_filter.nlargest(3,'Last_method',keep="first")
else:
    signal_output = signal_input_filter.sort_values(by="start_date").shift(1)

    signal_input_filter.sort_values(by='start_date',inplace=True)
    signal_input_filter['model_selected'] = signal_input_filter['model'].shift(1)
    
    signal_input_filter.to_excel('/Users/dylan/DataBase/signal_process/1.xlsx',index=False) """


' 当期相同则计算上期method\nif len(signal_input_filter[\'Max_method\']) > 3:       \n    signal_input_filter.sort_values(by=[\'model\',\'start_date\'],inplace=True) \n    signal_input_filter[\'Last_method\'] = signal_input_filter[\'Max_method\'].shift(1) \n    signal_output = signal_input_filter.nlargest(3,\'Last_method\',keep="first")\nelse:\n    signal_output = signal_input_filter.sort_values(by="start_date").shift(1)\n\n    signal_input_filter.sort_values(by=\'start_date\',inplace=True)\n    signal_input_filter[\'model_selected\'] = signal_input_filter[\'model\'].shift(1)\n    \n    signal_input_filter.to_excel(\'/Users/dylan/DataBase/signal_process/1.xlsx\',index=False) '

In [13]:
# 选出当期Top3

signal_input_filter = signal_input.groupby(by = 'start_date').apply(lambda x : x.nlargest(3, 'Max_method', keep = 'first'))

# 在Top3中随机选择
""" signal_output = pd.DataFrame()
for i in signal_input_filter['start_date'].drop_duplicates(): 
    column = signal_input_filter.loc[(i)].sample(1)
    signal_output = signal_output.append(column)

signal_output['model_selected'] = signal_output['model'].shift(1) """

# 对Top3 进行处理 综合信号




" signal_output = pd.DataFrame()\nfor i in signal_input_filter['start_date'].drop_duplicates(): \n    column = signal_input_filter.loc[(i)].sample(1)\n    signal_output = signal_output.append(column)\n\nsignal_output['model_selected'] = signal_output['model'].shift(1) "

In [11]:
# 归一化
""" for m in signal_input_filter['start_date'].drop_duplicates():
    for n in signal_input_filter['start_date'].drop_duplicates():
    M1 = signal_input_filter.loc[i].rename(columns = {'annual_return_sp' : 'a','win_rate_sp' : 'b','pc_ratio_sp' : 'c'}, inplace = False)
    M2 = signal_input_filter.loc[i].rename(columns = {'annual_return_sp_weight' : 'a','win_rate_sp_weight' : 'b','pc_ratio_sp_weight' : 'c'}, inplace = False)
    signal_output = M1[['a','b','c']].dot(M2[['a','b','c']].T).T.iloc[0]
    Max = signal_output.max()
    Min = signal_output.min()
    signal_output = (signal_output - Min)/(Max - Min)
    signal_input['trade_date' > m and 'trade_date' < n ]['top_annual_return_sp-30','top_win_rate_sp-30','top_pc_ratio_sp-30']
 """
                                                                                        
M1 = signal_input_filter.loc['2015-07-01'].rename(columns = {'annual_return_sp' : 'a','win_rate_sp' : 'b','pc_ratio_sp' : 'c'}, inplace = False)

M2 = signal_input_filter.loc['2015-07-01'].rename(columns = {'annual_return_sp_weight' : 'a','win_rate_sp_weight' : 'b','pc_ratio_sp_weight' : 'c'}, inplace = False)

signal_output = M1[['a','b','c']].dot(M2[['a','b','c']].T).T.iloc[0]
Max = signal_output.max()
Min = signal_output.min()
signal_output = (signal_output - Min)/(Max - Min)

signal_input[('trade_date' > '2015-07-01') and ('trade_date' < '2015-08-11')]
""" ['top_annual_return_sp-30','top_win_rate_sp-30','top_pc_ratio_sp-30'] """


KeyError: False

In [ ]:
### 得到信号

        
df1 = signal_output[['annual_return_sp','win_rate_sp','pc_ratio_sp']]
df3 = pd.DataFrame()
for x in signal_output['model']:
    df2 = signal_output[(x == signal_input['model'])]
    for y in signal_output['start_date']: 
        new_column = df2[y == df2['start_date']]
        df3 = df3.append(new_column)

df4 = df3.drop_duplicates('start_date')
df4.rename(columns = {'annual_return_sp_weight' : 'a','win_rate_sp_weight' : 'b','pc_ratio_sp_weight' : 'c'}, inplace = True)
df1.rename(columns = {'annual_return_sp' : 'a','win_rate_sp' : 'b','pc_ratio_sp' : 'c'}, inplace = True)
signal_output['signal'] = df1.dot(df4[['a','b','c']].T)

Max = signal_output['signal'].max()
Min = signal_output['signal'].min()
signal_output['signal'] = (signal_output['signal'] - Min)/(Max - Min)

/